In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from matplotlib import pyplot
from matplotlib.pyplot import figure

from sklearn.model_selection import train_test_split
random_state = 1
train_size = 0.75

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

# Save the data in a dataframe and cleaning

In [ ]:
# Read data from file (or url) and save the dataframe
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
df = pd.read_csv(url, sep = ';') 
# if the names of the columns are not present, insert them using `names = []`
# if the file is an excel use df = pd.read_excel(data_fn)
print(f"Shape of the input data {df.shape}")

target = 'quality'

In [ ]:
# showing the name of the columns
print(df.columns)
# showing the first rows of the dataframe
df.head()

In [ ]:
# printing the unique class labels
classes = df[target].unique()
classes.sort()
print(classes)

In [ ]:
# Generating descriptive statistics of the dataframe
# it's also possible to have description of a single feature using 
# df['feature'].describe()
df.describe(include='all')

In [ ]:
# Deleting the rows with null values
print(f"Number of rows with null values: {df.shape[0] - df.dropna().shape[0]}")
df1 = df.dropna().copy()
print(f"After dropping rows with nulls the dataset has {df1.shape[0]} rows")

In [ ]:
# some descriptions represent the same item but have different leading or trailing spaces,
# therefore they must be made uniform with the Pandas' function `str.strip()`
df1 = df
df1['Description'] = df['Description'].str.strip()

In [ ]:
print("Rows with missing InvoiceNo before removing")
df1[df1['InvoiceNo'].isna()]

df2 = df1.dropna(axis=0, subset=['InvoiceNo'])

print("Rows with missing InvoiceNo after removing")
df2[df2['InvoiceNo'].isna()]

In [ ]:
# deleting the column with a C in InvoiceNo
# in order to be able to use string functions, such as `contains`, 
# the column must be transformed into `str` with `astype`.
print(f"There are {sum(df2['InvoiceNo'].astype('str').str.contains('C'))} rows containing 'C' in 'InvoiceNo'")

df3 = df2[~df2['InvoiceNo'].astype('str').str.contains('C')]

print("After removal, there are {} rows containing 'C' in 'InvoiceNo'"\
      .format(sum(df3['InvoiceNo'].astype('str').str.contains('C'))))

In [ ]:
# Actions:
# 1. filter the rows ``Country`='France'`
# 2. group by `['InvoiceNo', 'Description']` computing a sum on `['Quantity']`
# 3. use the `unstack` function to move the items from rows to columns
# 4. reset the index
# 5. fill the missing with zero (`fillna(0)`)
# 6. store the result in the new dataframe `basket` and inspect it
basket = (df3[df3['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket

In [ ]:
# There are a lot of zeros in the data but we also need to make sure any 
# positive values are converted to a 1 and anything less than 0 is set to 0.
encode_units = lambda x: 0 if x <= 0 else 1
basket_sets = basket.applymap(encode_units)

In [ ]:
# Data transformation
# convert an alphanumeric number into numeric 0 and 1
df1['SexHRP']=df1['SexHRP'].apply(lambda x: 0 if (x=='Female') else 1)
# generate two new columns
df1['qmeat_hhsize_ratio'] = df1['qmeat']/df1['hhsize']
df1['income_hhsize_ratio'] = df1['income']/df1['hhsize']

# use only the most important feature
df = df1[['adults_n', 'children_n', 'SexHRP', 'AgeHRP'
        , 'qmeat_hhsize_ratio', 'income_hhsize_ratio', 'uvmeat']]

In [ ]:
# 3. Converting the attribute 'a3' to numeric
enc = OrdinalEncoder()
df[['a3']] = enc.fit_transform(df[['a3']])
df.head()

# Plots

In [ ]:
# showing the histogram of the dataset
# it's also possible to have the histogram of a single feature using 
# plt.hist(df['quality']) and plt.show()
pd.DataFrame.hist(df
                  , figsize = [10,10]
                 );

In [ ]:
# Plotting pairwise relationships in the dataframe
sns.pairplot(df, hue=target) #, height=2) # diag_kws={'bw': 0.2}

In [ ]:
# Computing pairwise correlation of columns
corr = df[df.columns].corr()
sns.heatmap(corr, cmap="YlGnBu", annot=True)

In [ ]:
# Checking outliers and observing the data distribution

# plt.figure(figsize=(15,15))
# pos = 1
# for i in df.columns:
#     plt.subplot(3, 4, pos)
#     sns.boxplot(df[i])
#     pos += 1

numCol = len(df.columns)
a1 = int(math.sqrt(numCol))
a2 = int(numCol/a1)
if (numCol%a1)>0:
    a2 += 1

plt.figure(figsize=(15,15))
pos = 1
for i in df.columns:
    plt.subplot(a1, a2, pos)
    sns.boxplot(df[i])
    pos += 1

In [ ]:
# boxplot to check the outliers (using this we have only one plot)
sns.boxplot(data = df)
sns.boxplot(x='quality', y='fixed acidity', data = df)

In [ ]:
# Showing the two dimensional scatter plots for all 
# the predicting variables with respect to the target
ncols=3
nrows = math.ceil((df.shape[1]-1)/ncols)
figwidth = ncols * 7
figheigth = nrows*5

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheigth),sharey=True)
plt.subplots_adjust(hspace=0.5)
fig.suptitle("Predicting variables versus target", fontsize=18, y=0.95)

for c, ax in zip(df.drop(target,axis=1).columns,axs.ravel()):
    df.sort_values(by=c).plot.scatter(x=c, y=target, \
        title = '"{}" versus "{}"'.format(target,c), ax=ax)

# splitting the data

In [ ]:
# storing in X the content of the dataframe excluding the target column
X = df.drop(target, axis=1)
# storing in y the labels
y = df[target]
print(f"Shape of X: {X.shape}\nShape of y: {y.shape}")

In [ ]:
# dividing the dataset in train and test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=random_state, train_size = train_size)
print("There are {} samples in the training dataset".format(Xtrain.shape[0]))
print("There are {} samples in the testing dataset".format(Xtest.shape[0]))
print("Each sample has {} features".format(Xtrain.shape[1]))

## operation for clustering
If plotting using the pairplot we see that some data are highhly concentrated on a side of the plot, we could apply some operation (for example the square root).
Then we could remap all the variables in the range 0:1 so that clistering is more effective.

In [ ]:
X_sqrt = pd.concat([df.iloc[:,:2],df.iloc[:,2:].applymap(math.sqrt)],axis=1)

# remap on the 0:1 range with MinMaxScaler
mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X_sqrt), columns = X_sqrt.columns)
X.head()

# show the transformation
X.boxplot(figsize=(15,8));
plt.show()
sns.pairplot(X);
plt.show()